In [146]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [147]:
covid = pd.read_csv('Covid19.csv', parse_dates= ['Date_reported'])

In [148]:
covid.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-04,NE,Niger,AFR,NaN,0,NaN,0
1,2020-01-04,NO,Norway,EUR,NaN,0,NaN,0
2,2020-01-04,PW,Palau,WPR,0.0,0,0.0,0
3,2020-01-04,PY,Paraguay,AMR,NaN,0,NaN,0
4,2020-01-04,PN,Pitcairn,WPR,0.0,0,0.0,0


In [149]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487680 entries, 0 to 487679
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Date_reported      487680 non-null  datetime64[ns]
 1   Country_code       485648 non-null  object        
 2   Country            487680 non-null  object        
 3   WHO_region         487680 non-null  object        
 4   New_cases          210274 non-null  float64       
 5   Cumulative_cases   487680 non-null  int64         
 6   New_deaths         153909 non-null  float64       
 7   Cumulative_deaths  487680 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 29.8+ MB


In [150]:
covid['New_cases'].isna().sum()

277406

<h3>56% Of data is missing</h3>

In [151]:
covid = covid.sort_values(["Country", "WHO_region", "Date_reported"]).reset_index(drop = True)

In [152]:
grouped_data = covid.groupby(['Country'	,'WHO_region'])

<h3>Sorting and grouping data for preparing for shifting operations</h3>

In [153]:
covid["Cumulative_cases_prev"] = grouped_data["Cumulative_cases"].shift(1, fill_value= 0)
covid["Cumulative_deaths_prev"] = grouped_data["Cumulative_deaths"].shift(1, fill_value= 0)

<h3>Shifting the data of the cumulative columns  to create a new column indicating the previous cumulative data</h3>

In [154]:
covid.isna().sum()

Date_reported                  0
Country_code                2032
Country                        0
WHO_region                     0
New_cases                 277406
Cumulative_cases               0
New_deaths                333771
Cumulative_deaths              0
Cumulative_cases_prev          0
Cumulative_deaths_prev         0
dtype: int64

In [155]:
covid['New_cases'] = covid.apply(
    lambda row: (row['Cumulative_cases'] - row['Cumulative_cases_prev']) if pd.isna(row['New_cases']) else row['New_cases'], axis= 1
)

<h3>Subtracting the previous cumulative cases to the current cumulative cases to fill the missing new cases values</h3>

In [156]:
covid['New_deaths'] = covid.apply(
    lambda row: (row['Cumulative_deaths'] - row['Cumulative_deaths_prev']) if pd.isna(row['New_deaths']) else row['New_deaths'], axis= 1
)

<h3>Subtracting the previous cumulative cases to the current cumulative cases to fill the missing new cases values</h3>

In [157]:
covid.dropna(inplace= True)

<h3>Dropping the missing Columns from country code columns since it's less than 5% o the data</h3>

In [158]:
covid.isna().sum()

Date_reported             0
Country_code              0
Country                   0
WHO_region                0
New_cases                 0
Cumulative_cases          0
New_deaths                0
Cumulative_deaths         0
Cumulative_cases_prev     0
Cumulative_deaths_prev    0
dtype: int64